# Auxiliary Classifier GAN

ref. ODENA, Augustus; OLAH, Christopher; SHLENS, Jonathon. 
     Conditional image synthesis with auxiliary classifier gans. 
     In: Proceedings of the 34th International Conference on Machine Learning-Volume 70. JMLR. org, 2017. p. 2642-2651.
     https://arxiv.org/pdf/1610.09585.pdf%5D

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [3]:
def build_generator(latent_dim, channels, num_classes):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

# Discriminator

In [4]:
def build_discriminator(img_shape, num_classes):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.summary()

    img = Input(shape=img_shape)

    # Extract feature representation
    features = model(img)

    # Determine validity and label of the image
    validity = Dense(1, activation="sigmoid")(features)
    label = Dense(num_classes, activation="softmax")(features)

    return Model(img, [validity, label])

# Autiliary functions

In [5]:
def sample_images(epoch, latent_dim, G):
    r, c = 10, 10
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    sampled_labels = np.array([num for _ in range(r) for num in range(c)])
    gen_imgs = G.predict([noise, sampled_labels])
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

# perform the training

In [6]:
def train(G, D, combined, epochs, latent_dim, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Configure inputs
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # The labels of the digits that the generator tries to create an
        # image representation of
        sampled_labels = np.random.randint(0, 10, (batch_size, 1))

        # Generate a half batch of new images
        gen_imgs = G.predict([noise, sampled_labels])

        # Image labels. 0-9 
        img_labels = y_train[idx]

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, [valid, img_labels])
        d_loss_fake = D.train_on_batch(gen_imgs, [fake, sampled_labels])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator
        g_loss = combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            save_model(G, D)
            sample_images(epoch, latent_dim, G)

In [7]:
def save(model, model_name, path="saved_model"):
    if not os.path.exists(path):
        os.makedirs(path)
    model_path = "{}/{}.json".format(path, model_name)
    weights_path = "{}/{}_weights.hdf5".format(path, model_name)
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [8]:
def save_model(G, D):
    save(G, "generator")
    save(D, "discriminator")

# main()

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100

In [10]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# type of losses used in the GAN
losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

In [12]:
# Build and compile the discriminator
D = build_discriminator(img_shape, num_classes)
D.compile(loss=losses,
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)         

In [13]:
# Build the generator
G = build_generator(latent_dim, channels, num_classes)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)      

In [14]:
# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = G([noise, label])

In [15]:
# For the combined model we will only train the generator
D.trainable = False

In [16]:
# The discriminator takes generated image as input and determines validity
# and the label of that image
valid, target_label = D(img)

In [17]:
# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model([noise, label], [valid, target_label])
combined.compile(loss=losses,
    optimizer=optimizer)

## run

In [ ]:
# epochs=14000
epochs=8000
train(G, D, combined, 
      latent_dim=latent_dim, 
      epochs=epochs, 
      batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.249759, acc.: 45.31%, op_acc: 12.50%] [G loss: 3.326734]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 3.344712, acc.: 53.12%, op_acc: 6.25%] [G loss: 3.432879]
2 [D loss: 3.087780, acc.: 68.75%, op_acc: 14.06%] [G loss: 3.092127]
3 [D loss: 2.893310, acc.: 81.25%, op_acc: 7.81%] [G loss: 3.170537]
4 [D loss: 2.807222, acc.: 95.31%, op_acc: 6.25%] [G loss: 3.010979]
5 [D loss: 2.846573, acc.: 92.19%, op_acc: 7.81%] [G loss: 3.021794]
6 [D loss: 2.868686, acc.: 96.88%, op_acc: 10.94%] [G loss: 2.880062]
7 [D loss: 2.604775, acc.: 96.88%, op_acc: 10.94%] [G loss: 2.797650]
8 [D loss: 2.692742, acc.: 98.44%, op_acc: 12.50%] [G loss: 2.837114]
9 [D loss: 2.473167, acc.: 100.00%, op_acc: 17.19%] [G loss: 2.973436]
10 [D loss: 2.544026, acc.: 98.44%, op_acc: 12.50%] [G loss: 2.717841]
11 [D loss: 2.676387, acc.: 100.00%, op_acc: 12.50%] [G loss: 2.948655]
12 [D loss: 2.546755, acc.: 98.44%, op_acc: 17.19%] [G loss: 2.770860]
13 [D loss: 2.638950, acc.: 100.00%, op_acc: 20.31%] [G loss: 2.817285]
14 [D loss: 2.455533, acc.: 100.00%, op_acc: 26.56%] [G loss: 2.883173]
15 [D loss: 2.6